In [1]:
import random
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import torch,torchvision
from torch.nn import *
from torch.optim import *
from sklearn.preprocessing import (
    StandardScaler,
    RobustScaler,
    MinMaxScaler,
    MaxAbsScaler,
    OneHotEncoder,
    LabelEncoder,
    Normalizer,
)
from sklearn.model_selection import cross_val_score,train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression,LogisticRegressionCV
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor,AdaBoostRegressor,VotingRegressor,BaggingRegressor,RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from catboost import CatBoost,CatBoostRegressor
from xgboost import XGBRegressor,XGBRFRegressor
from flaml import AutoML
from sklearn.metrics import mean_absolute_error,mean_squared_error,accuracy_score,precision_score,f1_score,recall_score
import pickle
import wandb
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GridSearchCV
PROJECT_NAME = 'House-Prices-Advanced-Regression-Techniques-V7'
device = 'cuda'
np.random.seed(21)
random.seed(21)
torch.manual_seed(21)

In [2]:
def make_submission(model,name):
    data = pd.read_csv('./data/test.csv')
    ids = data['Id']
    for one_hot_encoding_col in one_hot_encoding:
        data,idx,labels_and_int_index,new_data = object_to_int(data,one_hot_encoding_col)
    for num_of_samples,col,dtype in zip(data.isna().sum(),data.columns,data.dtypes):
        if dtype == 'object':
            pass
        else:
            if num_of_samples > 0:
                data[col].fillna(data[col].median(),inplace=True)
    
    preds = model.predict(data)
    df = pd.DataFrame({'Id':ids,'SalePrice':preds})
    df.to_csv(f'./submisssions/{name}.csv',index=False)
    return df
def validate(model,X,y,val=False):
    preds = model.predict(X)
    try:
        if val:
            result = {
                'MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'MSE':mean_squared_error(y_pred=preds,y_true=y),
                'Accuracy':model.score(X,y),
            }
        else:
            result = {
                'Val MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'Val MSE':mean_squared_error(y_pred=preds,y_true=y),
                'Val Accuracy':model.score(X,y),
            }
    except:
        if val:
            result = {
                'MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'MSE':mean_squared_error(y_pred=preds,y_true=y),
            }
        else:
            result = {
                'Val MAE':mean_absolute_error(y_pred=preds,y_true=y),
                'Val MSE':mean_squared_error(y_pred=preds,y_true=y),
            }
    return result
def train(model,X_train,X_test,y_train,y_test,name):
    wandb.init(project=PROJECT_NAME,name=name)
    model.fit(X_train,y_train)
    wandb.log(validate(model,X_train,y_train))
    wandb.log(validate(model,X_test,y_test,True))
    wandb.sklearn.plot_regressor(model, X_train, X_test, y_train, y_test,  model_name=name)
    pickle.dump(model,open(f'./models/model-{name}.pkl','wb'))
    make_submission(model,name)
    return model

In [3]:
data = pd.read_csv('./data/train.csv')

In [4]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [5]:
one_hot_encoding = []

In [6]:
for col_name,dtype in zip(data.columns,data.dtypes):
    if dtype == 'object':
        one_hot_encoding.append(col_name)

In [7]:
len(one_hot_encoding)

43

In [8]:
one_hot_encoding_data = data[one_hot_encoding]

In [9]:
def object_to_int(data,col):
    data_col = data[col].to_dict()
    idx = -1
    labels_and_int_index = {}
    for data_col_vals in data_col.values():
        if data_col_vals not in labels_and_int_index.keys():
            idx += 1
            labels_and_int_index[data_col_vals] = idx
    new_data = []
    for data_col_vals in data_col.values():
        new_data.append(labels_and_int_index[data_col_vals])
    data[col] = new_data
    return data,idx,labels_and_int_index,new_data

In [10]:
labels_and_int_indexs = {}

In [11]:
for one_hot_encoding_col in one_hot_encoding:
    data,idx,labels_and_int_index,new_data = object_to_int(data,one_hot_encoding_col)
    labels_and_int_index[one_hot_encoding_col] = labels_and_int_index

In [12]:
for num_of_samples,col,dtype in zip(data.isna().sum(),data.columns,data.dtypes):
    if dtype == 'object':
        pass
    else:
        if num_of_samples > 0:
            data[col].fillna(data[col].median(),inplace=True)

In [13]:
data.isna().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
SalePrice        0
Length: 81, dtype: int64

In [14]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0,65.0,8450,0,0,0,0,0,...,0,0,0,0,0,2,2008,0,0,208500
1,2,20,0,80.0,9600,0,0,0,0,0,...,0,0,0,0,0,5,2007,0,0,181500
2,3,60,0,68.0,11250,0,0,1,0,0,...,0,0,0,0,0,9,2008,0,0,223500
3,4,70,0,60.0,9550,0,0,1,0,0,...,0,0,0,0,0,2,2006,0,1,140000
4,5,60,0,84.0,14260,0,0,1,0,0,...,0,0,0,0,0,12,2008,0,0,250000


In [15]:
X = data.drop('SalePrice',axis=1)
y = data['SalePrice']

In [16]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125)

In [17]:
# model = GradientBoostingRegressor()
# train(model,X_train,X_test,y_train,y_test,f'First test GradientBoostingRegressor')

In [18]:
X_train_old = X_train.copy()
X_test_old = X_test.copy()

In [19]:
pres = [
#         StandardScaler,
#     RobustScaler,
#     MinMaxScaler,
#     MaxAbsScaler,
#     Normalizer,]
]

In [20]:
# StandardScaler = 1.2
# RobustScaler = 1.2
# MinMaxScaler = 1.1
# MaxAbsScaler = 1.2
# Normalizer = 0.47
# Normal = 0.15

In [21]:
# for pre in pres:
#     X_train = X_train_old.copy()
#     X_test = X_test_old.copy()
#     mct = make_column_transformer(
#         (pre(),X.columns)
#     )
#     X_train = mct.fit_transform(X_train)
#     X_test = mct.transform(X_test)
#     model = RandomForestRegressor()
#     train(model,X_train,X_test,y_train,y_test,f'Pre Proccessing - {pre()}')
# mct = make_column_transformer(
#     (pre(),X.columns)
# )
# X_train = mct.fit_transform(X_train)
# X_test = mct.transform(X_test)

In [22]:
models = [
#     ['KNeighborsRegressor',KNeighborsRegressor],
#     ['LogisticRegression',LogisticRegression],
#     ['LogisticRegressionCV',LogisticRegressionCV],
#     ['DecisionTreeRegressor',DecisionTreeRegressor],
#     ['GradientBoostingRegressor',GradientBoostingRegressor],
#     ['AdaBoostRegressor',AdaBoostRegressor],
#     ['RandomForestRegressor',RandomForestRegressor],
#     ['BaggingRegressor',BaggingRegressor],
#     ['GaussianNB',GaussianNB],
#     ['ExtraTreesRegressor',ExtraTreesRegressor],
#     ['CatBoost',CatBoost],
#     ['CatBoostRegressor',CatBoostRegressor],
#     ['XGBRegressor',XGBRegressor],
#     ['XGBRFRegressor',XGBRFRegressor],
#     ['ExtraTreesRegressor',ExtraTreesRegressor],
]

In [23]:
# for model in models:
#     train(model[1](),X_train,X_test,y_train,y_test,model[0])

In [24]:
# Random Forrest Regressor = 0.15
# Ada Boost Regressor = 0.21
# Decision Tree Regressor = 0.23
# Gradient Boost Regressor = 0.14
# BaggingRegressor = 0.16
# ExtraTreesRegressor = 0.16
# CatBoost = 0.15
# CatBoostRegressor = 0.16
# XGBRegressor = 0.16
# XGBRFRegressor = 

In [25]:
# grid_search = {
#     'loss':['ls','lad','huber','quantile'],
#     'learning_rate':[0.1,0.01,0.001,0.0001,0.00001],
#     'n_estimators':[100,125,250,500],
#     'criterion':['friedman_mse','mse','mae'],
#     'max_features':['auto','sqrt','log2'],
    
# }
# model = GridSearchCV(GradientBoostingRegressor(),verbose=5,cv=2,param_grid=grid_search)
# model.fit(X_train,y_train)

In [26]:
# model.best_params_

NameError: name 'model' is not defined

In [ ]:
# model = GradientBoostingRegressor(
# loss='ls',
# learning_rate=0.1,
# n_estimators=125,
# criterion='mse',
# max_features='sqrt'
# )
# train(model,X_train,X_test,y_train,y_test,'Final')

In [ ]:
model = XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 2000,
 max_depth= 4,
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1)
train(model,X_train,X_test,y_train,y_test,'Final-1')